# Guided Project: CIA World Facebook 

# Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The factbook contains demographic information like the following:

- `population` — the global population.
- `population_growth` - the annual population growth rate, as a percentage.  
- `area` — the total land and water area.

In this guided project, we'll use SQL in Jupyter Notebook to analyze data from this database. 

In [1]:
#lets connect to the datasource
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Part 1: Data Exploration

In [8]:
%%sql
SELECT *
  FROM facts limit 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's waterarea in square kilometers.
- `population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

In [10]:
%%sql
select min(population) as "Minimum population",
    max(population) as "Maximum Population",
    min(population_growth) as "Minimum population growth",
    max(population_growth) as "Maximum population growth"
from facts

Done.


Minimum population,Maximum Population,Minimum population growth,Maximum population growth
0,7256490011,0.0,4.02


# Part 2: Looking at Outliers

In [24]:
%%sql
select name from facts
where population == (select min(population) from facts)

Done.


name
Antarctica


In [25]:
%%sql
select name from facts
where population == (select max(population) from facts)

Done.


name
World


The table contains a row for the whole world, which explains the population of over 7.2 billion. 
The table contains a row for Antarctica, which explains the population of 0.

#### Let's exclude the world from data and observe summary statistics again

In [28]:
%%sql
select min(population) as "Minimum population",
    max(population) as "Maximum Population",
    min(population_growth) as "Minimum population growth",
    max(population_growth) as "Maximum population growth"
from facts
where name != 'World';

Done.


Minimum population,Maximum Population,Minimum population growth,Maximum population growth
0,1367485388,0.0,4.02


# Part 3: Finding Countries Above Pop & Area Avg

In [29]:
%%sql
select avg(population) as 'Population Average', avg(area) as 'Areaa Average'
from facts

Done.


Population Average,Areaa Average
62094928.32231405,555093.546184739


In [31]:
%%sql
select name as "Countries" from facts
where population > (select avg(population) from facts) 
and area > (select avg(area) from facts)

Done.


Countries
Brazil
China
"Congo, Democratic Republic of the"
Egypt
Ethiopia
France
India
Indonesia
Iran
Mexico


# Other questions to explore:

#### Which country has the most people? Which country has the highest growth rate?

In [37]:
%%sql
select name as "Country with the most people"
from facts 
where population == (select max(population) from facts
                    where name <> 'World')

Done.


Country with the most people
China


In [38]:
%%sql
select name as "Country that has highest growth rate"
from facts 
where population_growth == (select max(population_growth) from facts
                    where name <> 'World')

Done.


Country that has highest growth rate
South Sudan


#### Which countries have the highest ratios of water to land? Which countries have more water than land?

In [40]:
%%sql
select name as "Countries that have the highest ratios of water to land" 
from (select name, area_water/area_land from facts order by area_water/area_land desc)
limit 5

Done.


Countries that have the highest ratios of water to land
British Indian Ocean Territory
Virgin Islands
Afghanistan
Albania
Algeria


In [41]:
%%sql
select name as "Countries that have more water than land"
from facts where (area_water > area_land)

Done.


Countries that have more water than land
British Indian Ocean Territory
Virgin Islands


#### Which countries will add the most people to their populations next year?

In [42]:
%%sql
select name from facts
where (population_growth > death_rate)

Done.


name
Qatar
United Arab Emirates


#### Which countries have a higher death rate than birth rate?

In [49]:
%%sql 
select name as "Countries that have higher death rate than birth rate"
from facts where (death_rate > birth_rate)
order by (death_rate > birth_rate) desc

Done.


Countries that have higher death rate than birth rate
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary


#### Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [50]:
%%sql
select name as "Countries that have the highest population/area ratio"
from (select name, population/area from facts order by population/area desc)
limit 10

Done.


Countries that have the highest population/area ratio
Macau
Monaco
Singapore
Hong Kong
Gaza Strip
Gibraltar
Bahrain
Maldives
Malta
Bermuda
